In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from collections import Counter

In [55]:
df_raw = pd.read_csv("compas-scores-raw.csv")

In [56]:
print len(df_raw.columns)
df_raw.head()

28


,Person_ID,AssessmentID,Case_ID,Agency_Text,LastName,FirstName,MiddleName,Sex_Code_Text,Ethnic_Code_Text,DateOfBirth,...,RecSupervisionLevel,RecSupervisionLevelText,Scale_ID,DisplayText,RawScore,DecileScore,ScoreText,AssessmentType,IsCompleted,IsDeleted
0,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,12/05/92,...,1,Low,7,Risk of Violence,-2.08,4,Low,New,1,0
1,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,12/05/92,...,1,Low,8,Risk of Recidivism,-1.06,2,Low,New,1,0
2,50844,57167,51950,PRETRIAL,Fisher,Kevin,NaN,Male,Caucasian,12/05/92,...,1,Low,18,Risk of Failure to Appear,15.00,1,Low,New,1,0
3,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,09/16/84,...,1,Low,7,Risk of Violence,-2.84,2,Low,New,1,0
4,50848,57174,51956,PRETRIAL,KENDALL,KEVIN,NaN,Male,Caucasian,09/16/84,...,1,Low,8,Risk of Recidivism,-1.50,1,Low,New,1,0


In [57]:
df_scores = pd.read_csv("compas-scores.csv")
print len(df_scores.columns)
df_scores.iloc[:, 0:20].head()

47


,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_case_number,c_offense_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,13011352CF10A,2013-08-13
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,13001275CF10A,2013-01-26
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,13005330CF10A,2013-04-13
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,0,8,1,0,1,NaN,NaN,NaN,13000570CF10A,2013-01-12


In [58]:
df_scores.iloc[:, 20:35].head()

,c_arrest_date,c_days_from_compas,c_charge_degree,c_charge_desc,is_recid,num_r_cases,r_case_number,r_charge_degree,r_days_from_arrest,r_offense_date,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,num_vr_cases
0,NaN,1.0,F,Aggravated Assault w/Firearm,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN
1,NaN,NaN,O,NaN,-1,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN
2,NaN,1.0,F,Felony Battery w/Prior Convict,1,NaN,13009779CF10A,F,NaN,2013-07-05,Felony Battery (Dom Strang),NaN,NaN,1,NaN
3,NaN,1.0,F,Possession of Cocaine,1,NaN,13011511MM10A,M,0.0,2013-06-16,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN
4,NaN,1.0,F,Possession of Cannabis,0,NaN,NaN,O,NaN,NaN,NaN,NaN,NaN,0,NaN


In [59]:
df_scores.iloc[:, 35:].head()

,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,NaN,NaN,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,NaN,NaN,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,NaN,NaN,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,NaN,NaN,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13


In [60]:
type(df_scores['r_offense_date'][1])

float

In [61]:
#for i in range(df_scores.shape[0]):
for i in range(1):
    if df_scores['is_recid'][i] == 0 and df_scores['r_offense_date'][i] == 0:
        print i

In [62]:
print df_scores.columns

Index([u'id', u'name', u'first', u'last', u'compas_screening_date', u'sex',
       u'dob', u'age', u'age_cat', u'race', u'juv_fel_count', u'decile_score',
       u'juv_misd_count', u'juv_other_count', u'priors_count',
       u'days_b_screening_arrest', u'c_jail_in', u'c_jail_out',
       u'c_case_number', u'c_offense_date', u'c_arrest_date',
       u'c_days_from_compas', u'c_charge_degree', u'c_charge_desc',
       u'is_recid', u'num_r_cases', u'r_case_number', u'r_charge_degree',
       u'r_days_from_arrest', u'r_offense_date', u'r_charge_desc',
       u'r_jail_in', u'r_jail_out', u'is_violent_recid', u'num_vr_cases',
       u'vr_case_number', u'vr_charge_degree', u'vr_offense_date',
       u'vr_charge_desc', u'v_type_of_assessment', u'v_decile_score',
       u'v_score_text', u'v_screening_date', u'type_of_assessment',
       u'decile_score.1', u'score_text', u'screening_date'],
      dtype='object')


In [63]:
df_2 = pd.read_csv("compas-scores-two-years.csv")
df_2.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


In [64]:
len(df_2.columns)

53

In [65]:
df_violent = pd.read_csv("compas-scores-two-years-violent.csv")
df_violent.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,two_year_recid.1
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,1
2,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0,0
3,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,Low,2013-03-26,NaN,NaN,2,0,1102,0,0,0
4,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0,0


In [66]:
print len(df_violent.columns)

54


In [67]:
print set(df_2.columns) - set(df_scores.columns)
print set(df_scores.columns) - set(df_2.columns)
print set(df_violent.columns) - set(df_2.columns)
print set(df_2.columns) - set(df_violent.columns)

set(['end', 'two_year_recid', 'start', 'in_custody', 'out_custody', 'violent_recid', 'event', 'priors_count.1'])
set(['num_vr_cases', 'num_r_cases'])
set(['two_year_recid.1'])
set([])


Every single type-of-assessment entry is "Risk of recidivism", so we can drop this column

In [68]:
df_scores[df_scores['type_of_assessment'] != df_scores['type_of_assessment'][0]]

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date


We need to turn the 'c_charge_desc' and 'r_charge_desc' into binary variables

In [16]:
df_scores = pd.read_csv("compas-scores.csv")
lst = df_scores['c_charge_desc']
print len(lst)

11757


In [73]:
df_scores.head(n=10)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,vr_offense_date,vr_charge_desc,v_type_of_assessment,v_decile_score,v_score_text,v_screening_date,type_of_assessment,decile_score.1,score_text,screening_date
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-08-14,Risk of Recidivism,1,Low,2013-08-14
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-12-31,Risk of Recidivism,5,Medium,2014-12-31
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,2013-07-05,Felony Battery (Dom Strang),Risk of Violence,1,Low,2013-01-27,Risk of Recidivism,3,Low,2013-01-27
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,NaN,NaN,Risk of Violence,3,Low,2013-04-14,Risk of Recidivism,4,Low,2013-04-14
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,NaN,NaN,Risk of Violence,6,Medium,2013-01-13,Risk of Recidivism,8,High,2013-01-13
5,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-03-26,Risk of Recidivism,1,Low,2013-03-26
6,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,NaN,NaN,Risk of Violence,1,Low,2013-11-30,Risk of Recidivism,1,Low,2013-11-30
7,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,2,Low,2014-02-19,Risk of Recidivism,6,Medium,2014-02-19
8,9,steven stewart,steven,stewart,2013-08-30,Male,1973-02-25,43,25 - 45,Other,...,NaN,NaN,Risk of Violence,3,Low,2013-08-30,Risk of Recidivism,4,Low,2013-08-30
9,10,elizabeth thieme,elizabeth,thieme,2014-03-16,Female,1976-06-03,39,25 - 45,Caucasian,...,NaN,NaN,Risk of Violence,1,Low,2014-03-16,Risk of Recidivism,1,Low,2014-03-16


In [74]:
Counter(df_scores['c_charge_degree'])

Counter({'F': 7232, 'M': 3771, 'O': 754})

In [78]:
len(Counter(df_scores['c_charge_desc']).most_common())

532

In [77]:
Counter(df_scores['c_charge_desc']).most_common()[:40]

[('arrest case no charge', 1858),
 ('Battery', 1811),
 (nan, 749),
 ('Possession of Cocaine', 703),
 ('Grand Theft in the 3rd Degree', 688),
 ('Driving While License Revoked', 255),
 ('Driving Under The Influence', 217),
 ('Felony Battery (Dom Strang)', 174),
 ('Felony Driving While Lic Suspd', 159),
 ('Grand Theft (Motor Vehicle)', 139),
 ('Burglary Unoccupied Dwelling', 132),
 ('Pos Cannabis W/Intent Sel/Del', 131),
 ('Possess Cannabis/20 Grams Or Less', 127),
 ('Aggravated Assault W/Dead Weap', 124),
 ('Burglary Conveyance Unoccup', 124),
 ('DUI Property Damage/Injury', 111),
 ('Possession of Cannabis', 109),
 ('Aggrav Battery w/Deadly Weapon', 101),
 ('Felony Petit Theft', 98),
 ('DUI Level 0.15 Or Minor In Veh', 93),
 ('Driving License Suspended', 92),
 ('Uttering a Forged Instrument', 91),
 ('Resist Officer w/Violence', 90),
 ('Battery on Law Enforc Officer', 83),
 ('Poss3,4 Methylenedioxymethcath', 81),
 ('Possession Of Alprazolam', 79),
 ('Aggravated Battery / Pregnant', 75),
 

First we're going to append "c_" to all of the crime names and "r_" to all of the recidivism names

In [83]:
for i, j in df_scores.iterrows():
    if df_scores.loc[i, "c_charge_desc"] == 'Aggravated Assault W/dead Weap':
        df_scores.loc[i, "c_charge_desc"] = 'Aggravated Assault W/Dead Weap'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Grand Theft (motor vehicle)':
        df_scores.loc[i, "c_charge_desc"] = 'Grand Theft (Motor Vehicle)'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'DUI- Enhanced':
        df_scores.loc[i, "c_charge_desc"] = 'DUI - Enhanced'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'DUI- Enhanced':
        df_scores.loc[i, "c_charge_desc"] = 'DUI Property Damage'
        
    elif df_scores.loc[i, "c_charge_desc"] in ['DUI Property Damage', 'DUI/Property Damage/Persnl Inj', 'DUI - Property Damage/Personal Injury']: 
        df_scores.loc[i, "c_charge_desc"] = 'Property Damage/Injury'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Dealing In Stolen Property':
        df_scores.loc[i, "c_charge_desc"] = 'Dealing in Stolen Property'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Grand Theft In The 3Rd Degree':
        df_scores.loc[i, "c_charge_desc"] = 'Grand Theft in the 3rd Degree'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Opert With Susp DL 2ND Offense':
        df_scores.loc[i, "c_charge_desc"] = 'Opert With Susp DL 2nd Offens'
        
    elif df_scores.loc[i, "c_charge_desc"] == 'Offer Agree Secure For Lewd Act':
        df_scores.loc[i, "c_charge_desc"] = 'Offer Agree Secure/Lewd Act'
    
    elif df_scores.loc[i, "c_charge_desc"] in ['Poss Meth/Diox/Meth/Amp (MDMA)', 'Poss 3,4 Methylendioxpyrovlerone']: 
        df_scores.loc[i, "c_charge_desc"] = 'Poss 3,4 MDMA (Ecstasy)'
    
    elif df_scores.loc[i, "c_charge_desc"] in ['Trespass Other Struct/Conve', 'Trespass Struct/Conveyance', 'Tresspass Struct/Conveyance', 'Tresspass in Structure or Conveyance']: 
        df_scores.loc[i, "c_charge_desc"] = 'Trespass Structure/Conveyance'
    
    elif df_scores.loc[i, "c_charge_desc"] == 'Crim Use Of Personal Id Info':
        df_scores.loc[i, "c_charge_desc"] = 'Crim Use of Personal ID Info'
    
    elif df_scores.loc[i, "c_charge_desc"] == 'Crim Attempt/Solic/Consp':
        df_scores.loc[i, "c_charge_desc"] = 'Crim Attempt/Solicit/Consp'
    
    elif df_scores.loc[i, "c_charge_desc"] == 'Agg Fleeing And Eluding':
        df_scores.loc[i, "c_charge_desc"] = 'Agg Fleeing and Eluding'    

In [84]:
len(df_scores['c_charge_desc'].unique())

515

In [169]:
Counter(df_scores['c_charge_desc']).most_common()[0][0]

'arrest case no charge'

In [211]:
cats = {}
cats['battery'] = []
cats['assault'] = []
cats['drugs'] = []
cats['driving'] = []
cats['property'] = []
cats['weapon'] = []
cats['sexual'] = []
cats['obstruct'] = []
cats['domestic'] = []

nocat = []
for i in range(len(Counter(df_scores['c_charge_desc']).most_common())):
    charge = Counter(df_scores['c_charge_desc']).most_common()[i][0]
    crime = str(charge).lower()
    if 'batt' in crime:
        cats['battery'].append(charge)
    elif 'assault' in crime or 'abuse' in crime:
        cats['assault'].append(charge)
    elif 'poss' in crime or 'pos ' in crime or 'subst' in crime or 'cannabis' in crime or 'cocaine' in crime or 'deliver' in crime or 'del ' in crime:
        cats['drugs'].append(charge)
    elif 'dui' in crime or 'driv' in crime or 'veh' in crime or 'operat' in crime or ' dl' in crime or 'd.u.i' in crime or 'dwi' in crime:
        cats['driving'].append(charge)
    elif 'burg' in crime or 'robb' in crime or 'arson' in crime or 'theft' in crime or 'property' in crime or 'mischief' in crime or 'trespass' in crime:
        cats['property'].append(charge)
    elif 'firearm' in crime or 'weapon' in crime or 'arm' in crime:
        cats['weapon'].append(charge)
    elif 'sex' in crime or 'prostitu' in crime or 'lewd' in crime or 'voyeur' in crime or 'porn' in crime:
        cats['sexual'].append(charge)
    elif 'flee' in crime or 'forg' in crime or 'tamper' in crime or 'false' in crime or 'obstruct' in crime or 'obey' in crime or 'resist' in crime or ' id' in crime:
        cats['obstruct'].append(charge)
    elif 'dom' in crime or 'child' in crime:
        cats['domestic'].append(charge)
    else:
        nocat.append((charge, Counter(df_scores['c_charge_desc']).most_common()[i][1]))

In [212]:
print len(cats['battery'])
print len(cats['assault'])
print len(cats['drugs'])
print len(cats['driving'])
print len(cats['property'])
print len(cats['weapon'])
print len(cats['sexual'])
print len(cats['obstruct'])
print len(cats['domestic'])
print len(nocat)
print nocat[2:50]

35
15
136
37
67
15
24
35
9
142
[('Disorderly Conduct', 45), ('Viol Prot Injunc Repeat Viol', 33), ('Disorderly Intoxication', 20), ('Prowling/Loitering', 19), ('Leaving the Scene of Accident', 18), ('Corrupt Public Servant', 15), ('Exposes Culpable Negligence', 14), ('Unlaw LicTag/Sticker Attach', 12), ('Aggrav Stalking After Injunctn', 12), ('Fraudulent Use of Credit Card', 12), ('Stalking (Aggravated)', 11), ('Introduce Contraband Into Jail', 8), ('Ride Tri-Rail Without Paying', 5), ('Fail To Redeliv Hire/Leas Prop', 5), ('Sell or Offer for Sale Counterfeit Goods', 5), ('Alcoholic Beverage Violation-FL', 4), ('Use of Anti-Shoplifting Device', 4), ('Extradition/Defendants', 4), ('Solicitation On Felony 3 Deg', 4), ('Depriv LEO of Protect/Communic', 4), ('Threat Public Servant', 4), ('Stalking', 4), ('Refuse Submit Blood/Breath Test', 3), ('Throw Missile Into Pub/Priv Dw', 3), ('Escape', 3), ('Traffick Oxycodone     4g><14g', 3), ('Unemployment Compensatn Fraud', 3), ('Crim Attempt/Sol

In [213]:
cats = {}
cats['battery'] = []
cats['assault'] = []
cats['drugs'] = []
cats['driving'] = []
cats['property'] = []
cats['weapon'] = []
cats['sexual'] = []
cats['obstruct'] = []
cats['domestic'] = []

nocat = []
for i in range(len(Counter(df_scores['c_charge_desc']).most_common())):
    charge = Counter(df_scores['c_charge_desc']).most_common()[i][0]
    crime = str(charge).lower()
    if 'batt' in crime:
        cats['battery'].append(charge)
    if 'assault' in crime or 'abuse' in crime:
        cats['assault'].append(charge)
    if 'poss' in crime or 'pos ' in crime or 'subst' in crime or 'cannabis' in crime or 'cocaine' in crime or 'deliver' in crime or 'del ' in crime:
        cats['drugs'].append(charge)
    if 'dui' in crime or 'driv' in crime or 'veh' in crime or 'operat' in crime or ' dl' in crime or 'd.u.i' in crime or 'dwi' in crime:
        cats['driving'].append(charge)
    if 'burg' in crime or 'robb' in crime or 'arson' in crime or 'theft' in crime or 'property' in crime or 'mischief' in crime or 'trespass' in crime:
        cats['property'].append(charge)
    if 'firearm' in crime or 'weapon' in crime or 'arm' in crime:
        cats['weapon'].append(charge)
    if 'sex' in crime or 'prostitu' in crime or 'lewd' in crime or 'voyeur' in crime or 'porn' in crime:
        cats['sexual'].append(charge)
    if 'flee' in crime or 'forg' in crime or 'tamper' in crime or 'false' in crime or 'obstruct' in crime or 'obey' in crime or 'resist' in crime or ' id' in crime:
        cats['obstruct'].append(charge)
    if 'dom' in crime or 'child' in crime:
        cats['domestic'].append(charge)

In [214]:
print len(cats['battery'])
print len(cats['assault'])
print len(cats['drugs'])
print len(cats['driving'])
print len(cats['property'])
print len(cats['weapon'])
print len(cats['sexual'])
print len(cats['obstruct'])
print len(cats['domestic'])

35
21
137
42
77
51
30
44
24


In [223]:
np.histogram(df_scores['age'])

a18 = 0
a26 = 0
a34 = 0
a42 = 0
a50 = 0
a58 = 0
last = 0

for a in df_scores['age']:
    if a <= 18:
        a18 += 1
    elif a <= 24:
        a26 += 1
    elif a <= 29:
        a34 += 1
    elif a <= 34:
        a42 += 1
    elif a <= 39:
        a50 += 1
    elif a <= 50:
        a58 += 1
    else:
        last += 1
print a18, a26, a34, a42, a50, a58, last

7 2433 2453 1882 1352 1968 1662


# feature list

Categories are: 
battery, assault, driving, drugs, property, weapon-related, sexual, obstruction of justice/forgery, domestic violence, age less than 18, 19-24, 25-29, 30-34, 35-39, 40-50, 50+, c_felony, c_misdemeanor, (juv felony, misd, other = 1, >=2), priors count >= 1, >=3, >=5, >= 10

juvenile felony, juvenile misdemeanor, juvenile other, priors count, jail time

In [242]:
print np.histogram(df_scores['juv_fel_count'])
print np.histogram(df_scores['juv_misd_count'])
print np.histogram(df_scores['juv_other_count'])
print np.histogram(df_scores['priors_count'])

(array([11614,   110,    24,     2,     3,     2,     1,     0,     0,
           1]), array([ 0.,  2.,  4.,  6.,  8., 10., 12., 14., 16., 18., 20.]))
(array([11586,   100,    37,    21,     5,     2,     3,     0,     0,
           3]), array([ 0. ,  1.3,  2.6,  3.9,  5.2,  6.5,  7.8,  9.1, 10.4, 11.7, 13. ]))
(array([11550,   172,    26,     2,     3,     2,     1,     0,     0,
           1]), array([ 0. ,  1.7,  3.4,  5.1,  6.8,  8.5, 10.2, 11.9, 13.6, 15.3, 17. ]))
(array([9109, 1398,  603,  374,  139,   74,   46,    6,    6,    2]), array([ 0. ,  4.3,  8.6, 12.9, 17.2, 21.5, 25.8, 30.1, 34.4, 38.7, 43. ]))


In [256]:
print df_scores['c_jail_in'][2]
print df_scores['c_jail_out'][2]

2013-01-26 03:45:27
2013-02-05 05:36:53


In [260]:
from datetime import datetime

for i in range(2,10):
    start_date = df_scores['c_jail_in'][i]
    end_date = df_scores['c_jail_out'][i]
    start = __datetime(start_date)
    end = __datetime(end_date)
    diff = end - start
    print diff.total_seconds()/float(3600*24)

def __datetime(date_str):
    return datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S')


10.0773842593
1.08576388889


TypeError: strptime() argument 1 must be string, not float

In [90]:
for charge in sorted(df_scores['c_charge_desc'].unique()):
    if 'battery' in str(charge).lower():
        print charge

Agg Battery Cause Bodily
Agg Battery Grt/Bod/Harm
Aggrav Battery w/Deadly Weapon
Aggrav Child Abuse-Agg Battery
Aggravated Battery
Aggravated Battery (Firearm)
Aggravated Battery (Firearm/Actual Possession)
Aggravated Battery / Pregnant
Aggravated Battery On 65/Older
Assault/Battery On An Official
Attempted Aggravated Battery
Battery
Battery Emergency Care Provide
Battery On A Person Over 65
Battery On Fire Fighter
Battery On Parking Enfor Speci
Battery Spouse Or Girlfriend
Battery on Law Enforc Officer
Battery on a Person Over 65
Burglary Assault/Battery Armed
Burglary With Assault/battery
Domestic Battery By Stran
Felony Battery
Felony Battery (Dom Strang)
Felony Battery w/Prior Convict
Lewd/Lasc Battery Pers 12+/<16
Sex Battery Deft 18+/Vict 11-
Sexual Battery / Vict 12 Yrs +
Solic to Commit Battery


In [91]:
Counter(df_scores['c_charge_desc']).most_common()[:4]

[('arrest case no charge', 1858),
 ('Battery', 1811),
 (nan, 749),
 ('Possession of Cocaine', 703)]

In [17]:
df_scores['c_charge_desc'] = 'c_' + df_scores['c_charge_desc'].astype(str)

In [18]:
df_scores_binary = df_scores.join(pd.get_dummies(df_scores.pop('c_charge_desc')))

In [19]:
df_scores_binary.shape

(11757, 578)

In [20]:
df_scores_binary.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,c_Viol Pretrial Release Dom Viol,c_Viol Prot Injunc Repeat Viol,c_Violation License Restrictions,c_Violation Of Boater Safety Id,c_Violation of Injunction Order/Stalking/Cyberstalking,c_Voyeurism,c_Wear Mask w/Commit Offense,c_Workers Compensation Fraud,c_arrest case no charge,c_nan
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,0,0,0,0,0,0,0,0,0,0
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,0,0,0,0,0,0,0,0,0,1
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,0,0,0,0,0,0,0,0,0,0
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,0,0,0,0,0,0,0,0,0,0
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_temp = df_scores

In [22]:
df_temp['r_charge_desc'] = 'r_' + df_temp['r_charge_desc'].astype(str)

In [23]:
df_scores_binary_rc = df_scores_binary.join(pd.get_dummies(df_temp.pop('r_charge_desc')))

In [24]:
df_scores_binary_rc.shape

(11757, 931)

In [25]:
df_scores_binary_rc.drop(['r_nan'], axis=1).head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,r_Uttering Forged Bills,r_Uttering a Forged Instrument,r_Viol Injunct Domestic Violence,r_Viol Injunction Protect Dom Vi,r_Viol Injunction Protect Dom Violence,r_Viol Pretrial Release Dom Viol,r_Viol Prot Injunc Repeat Viol,r_Violation License Restrictions,r_Violation of Injunction Order/Stalking/Cyberstalking,r_Wear Mask w/Commit Offense
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,0,0,0,0,0,0,0,0,0,0
1,2,michael ryan,michael,ryan,2014-12-31,Male,1985-02-06,31,25 - 45,Caucasian,...,0,0,0,0,0,0,0,0,0,0
2,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,0,0,0,0,0,0,0,0,0,0
3,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,0,0,0,0,0,0,0,0,0,0
4,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,0,0,0,0,0,0,0,0,0,0


Above, df_scores_binary_rc has converted every c_charge_desc and r_charge_desc column into binary variables

In [26]:
df_dropped = df_scores_binary_rc.drop(['name', 'first', 'last', 'compas_screening_date', 'sex', 'dob', 'race', 'c_case_number', 'r_case_number',
                         'type_of_assessment', 'v_type_of_assessment', 'screening_date'], axis=1)

In [27]:
df_dropped.iloc[:, :20].head()

,id,age,age_cat,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,c_jail_out,c_offense_date,c_arrest_date,c_days_from_compas,c_charge_degree,is_recid,num_r_cases,r_charge_degree,r_days_from_arrest,r_offense_date
0,1,69,Greater than 45,0,1,0,0,0,-1.0,2013-08-13 06:03:42,2013-08-14 05:41:20,2013-08-13,NaN,1.0,F,0,NaN,O,NaN,NaN
1,2,31,25 - 45,0,5,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,O,-1,NaN,O,NaN,NaN
2,3,34,25 - 45,0,3,0,0,0,-1.0,2013-01-26 03:45:27,2013-02-05 05:36:53,2013-01-26,NaN,1.0,F,1,NaN,F,NaN,2013-07-05
3,4,24,Less than 25,0,4,0,1,4,-1.0,2013-04-13 04:58:34,2013-04-14 07:02:04,2013-04-13,NaN,1.0,F,1,NaN,M,0.0,2013-06-16
4,5,23,Less than 25,0,8,1,0,1,NaN,NaN,NaN,2013-01-12,NaN,1.0,F,0,NaN,O,NaN,NaN


In [28]:
df_dropped.shape

(11757, 919)

In [29]:
df_dropped.iloc[:, 20:40].head()

,r_charge_desc,r_jail_in,r_jail_out,is_violent_recid,num_vr_cases,vr_case_number,vr_charge_degree,vr_offense_date,vr_charge_desc,v_decile_score,v_score_text,v_screening_date,decile_score.1,score_text,c_Abuse Without Great Harm,c_Accessory After the Fact,c_Agg Abuse Elderlly/Disabled Adult,c_Agg Assault Law Enforc Officer,c_Agg Assault W/int Com Fel Dome,c_Agg Battery Cause Bodily
0,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,1,Low,2013-08-14,1,Low,0,0,0,0,0,0
1,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,2,Low,2014-12-31,5,Medium,0,0,0,0,0,0
2,Felony Battery (Dom Strang),NaN,NaN,1,NaN,13009779CF10A,(F3),2013-07-05,Felony Battery (Dom Strang),1,Low,2013-01-27,3,Low,0,0,0,0,0,0
3,Driving Under The Influence,2013-06-16,2013-06-16,0,NaN,NaN,NaN,NaN,NaN,3,Low,2013-04-14,4,Low,0,0,0,0,0,0
4,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,6,Medium,2013-01-13,8,High,0,0,0,0,0,0


In [30]:
df_dropped.iloc[:, 900:].head()

,r_Unnatural/Lascivious Act,r_Use Computer for Child Exploit,r_Use Fraud OBT Rcpt/False Rcpt,r_Use Lost Or Stolen Credit Card,r_Use Scanning Device to Defraud,r_Use of Anti-Shoplifting Device,r_Use/Poss/Drug Para/Plant/Grow,r_Utilizing Juvenile to Deliver,r_Uttering Forged Bills,r_Uttering a Forged Instrument,r_Viol Injunct Domestic Violence,r_Viol Injunction Protect Dom Vi,r_Viol Injunction Protect Dom Violence,r_Viol Pretrial Release Dom Viol,r_Viol Prot Injunc Repeat Viol,r_Violation License Restrictions,r_Violation of Injunction Order/Stalking/Cyberstalking,r_Wear Mask w/Commit Offense,r_nan
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


num_r_cases is always 0

In [31]:
df_dropmore = df_dropped.drop(['vr_case_number', 'r_charge_desc', 'num_r_cases', 'num_vr_cases', 'c_jail_in', 'c_jail_out', 
                 'c_arrest_date', 'r_days_from_arrest', 'r_offense_date', 'r_jail_in', 'r_jail_out', 
                 'vr_charge_degree', 'vr_offense_date', 'vr_charge_desc', 'v_decile_score', 'decile_score',
                'v_score_text', 'score_text', 'v_screening_date', 'c_offense_date'], axis=1)

In [32]:
df_dropmore.iloc[:, :20].head()

,id,age,age_cat,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,c_charge_degree,is_recid,r_charge_degree,is_violent_recid,decile_score.1,c_Abuse Without Great Harm,c_Accessory After the Fact,c_Agg Abuse Elderlly/Disabled Adult,c_Agg Assault Law Enforc Officer,c_Agg Assault W/int Com Fel Dome,c_Agg Battery Cause Bodily
0,1,69,Greater than 45,0,0,0,0,-1.0,1.0,F,0,O,0,1,0,0,0,0,0,0
1,2,31,25 - 45,0,0,0,0,NaN,NaN,O,-1,O,0,5,0,0,0,0,0,0
2,3,34,25 - 45,0,0,0,0,-1.0,1.0,F,1,F,1,3,0,0,0,0,0,0
3,4,24,Less than 25,0,0,1,4,-1.0,1.0,F,1,M,0,4,0,0,0,0,0,0
4,5,23,Less than 25,0,1,0,1,NaN,1.0,F,0,O,0,8,0,0,0,0,0,0


In [33]:
df_age_binary = df_dropmore.join(pd.get_dummies(df_dropmore.pop('age_cat')))

In [34]:
df_age_binary.iloc[:, :20]

,id,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,c_charge_degree,is_recid,r_charge_degree,is_violent_recid,decile_score.1,c_Abuse Without Great Harm,c_Accessory After the Fact,c_Agg Abuse Elderlly/Disabled Adult,c_Agg Assault Law Enforc Officer,c_Agg Assault W/int Com Fel Dome,c_Agg Battery Cause Bodily,c_Agg Battery Grt/Bod/Harm
0,1,69,0,0,0,0,-1.0,1.0,F,0,O,0,1,0,0,0,0,0,0,0
1,2,31,0,0,0,0,NaN,NaN,O,-1,O,0,5,0,0,0,0,0,0,0
2,3,34,0,0,0,0,-1.0,1.0,F,1,F,1,3,0,0,0,0,0,0,0
3,4,24,0,0,1,4,-1.0,1.0,F,1,M,0,4,0,0,0,0,0,0,0
4,5,23,0,1,0,1,NaN,1.0,F,0,O,0,8,0,0,0,0,0,0,0
5,6,43,0,0,0,2,NaN,76.0,F,0,O,0,1,0,0,0,0,0,0,0
6,7,44,0,0,0,0,0.0,0.0,M,0,O,0,1,0,0,0,0,0,0,0
7,8,41,0,0,0,14,-1.0,1.0,F,1,F,0,6,0,0,0,0,0,0,0
8,9,43,0,0,0,3,-1.0,1.0,F,0,O,0,4,0,0,0,0,0,0,0
9,10,39,0,0,0,0,-1.0,1.0,M,0,O,0,1,0,0,0,0,0,0,0


In [35]:
df_age_binary['c_charge_degree'] = 'c_' + df_age_binary['c_charge_degree'].astype(str)

In [36]:
df_age_binary['r_charge_degree'] = 'r_' + df_age_binary['r_charge_degree'].astype(str)

In [37]:
df_c_degree = df_age_binary.join(pd.get_dummies(df_age_binary.pop('c_charge_degree')))

In [38]:
df_c_degree.head()

,id,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,r_charge_degree,...,r_Violation License Restrictions,r_Violation of Injunction Order/Stalking/Cyberstalking,r_Wear Mask w/Commit Offense,r_nan,25 - 45,Greater than 45,Less than 25,c_F,c_M,c_O
0,1,69,0,0,0,0,-1.0,1.0,0,r_O,...,0,0,0,1,0,1,0,1,0,0
1,2,31,0,0,0,0,NaN,NaN,-1,r_O,...,0,0,0,1,1,0,0,0,0,1
2,3,34,0,0,0,0,-1.0,1.0,1,r_F,...,0,0,0,0,1,0,0,1,0,0
3,4,24,0,0,1,4,-1.0,1.0,1,r_M,...,0,0,0,0,0,0,1,1,0,0
4,5,23,0,1,0,1,NaN,1.0,0,r_O,...,0,0,0,1,0,0,1,1,0,0


In [39]:
df_r_degree = df_c_degree.join(pd.get_dummies(df_c_degree.pop('r_charge_degree')))

In [40]:
df_r_degree.head()

,id,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,...,r_nan,25 - 45,Greater than 45,Less than 25,c_F,c_M,c_O,r_F,r_M,r_O
0,1,69,0,0,0,0,-1.0,1.0,0,0,...,1,0,1,0,1,0,0,0,0,1
1,2,31,0,0,0,0,NaN,NaN,-1,0,...,1,1,0,0,0,0,1,0,0,1
2,3,34,0,0,0,0,-1.0,1.0,1,1,...,0,1,0,0,1,0,0,1,0,0
3,4,24,0,0,1,4,-1.0,1.0,1,0,...,0,0,0,1,1,0,0,0,1,0
4,5,23,0,1,0,1,NaN,1.0,0,0,...,1,0,0,1,1,0,0,0,0,1


In [41]:
df_c_degree

,id,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_days_from_compas,is_recid,is_violent_recid,...,r_Violation License Restrictions,r_Violation of Injunction Order/Stalking/Cyberstalking,r_Wear Mask w/Commit Offense,r_nan,25 - 45,Greater than 45,Less than 25,c_F,c_M,c_O
0,1,69,0,0,0,0,-1.0,1.0,0,0,...,0,0,0,1,0,1,0,1,0,0
1,2,31,0,0,0,0,NaN,NaN,-1,0,...,0,0,0,1,1,0,0,0,0,1
2,3,34,0,0,0,0,-1.0,1.0,1,1,...,0,0,0,0,1,0,0,1,0,0
3,4,24,0,0,1,4,-1.0,1.0,1,0,...,0,0,0,0,0,0,1,1,0,0
4,5,23,0,1,0,1,NaN,1.0,0,0,...,0,0,0,1,0,0,1,1,0,0
5,6,43,0,0,0,2,NaN,76.0,0,0,...,0,0,0,1,1,0,0,1,0,0
6,7,44,0,0,0,0,0.0,0.0,0,0,...,0,0,0,1,1,0,0,0,1,0
7,8,41,0,0,0,14,-1.0,1.0,1,0,...,0,0,0,0,1,0,0,1,0,0
8,9,43,0,0,0,3,-1.0,1.0,0,0,...,0,0,0,1,1,0,0,1,0,0
9,10,39,0,0,0,0,-1.0,1.0,0,0,...,0,0,0,1,1,0,0,0,1,0


In [42]:
df_dropped_nan = df_r_degree.dropna(axis=0, how='any')

In [43]:
cols = list(df_dropped_nan.columns)
cols.index('is_recid')

8

In [44]:
new_cols = []
new_cols.append(cols[8])
for i in range(len(cols)):
    if i != 8:
        new_cols.append(cols[i])

In [45]:
df_reorder = df_dropped_nan[new_cols]

In [46]:
df_reorder.columns

Index([u'is_recid', u'id', u'age', u'juv_fel_count', u'juv_misd_count',
       u'juv_other_count', u'priors_count', u'days_b_screening_arrest',
       u'c_days_from_compas', u'is_violent_recid',
       ...
       u'r_nan', u'25 - 45', u'Greater than 45', u'Less than 25', u'c_F',
       u'c_M', u'c_O', u'r_F', u'r_M', u'r_O'],
      dtype='object', length=905)

In [48]:
df_redrop = df_reorder.drop(['id', 'is_violent_recid'], axis=1)

In [49]:
df_redrop.to_csv('compas-no-nan-new.csv', index = False)

In [53]:
df_redrop.iloc[:, :50].to_csv('compas-no-nan-50.csv', index = False)